In [1]:
import pandas as pd
pd.set_option("display.max_columns", 101)

# Clean and prepare dataset

We create a dataframe named df_pivot that contains job features (skills and knowledge) as columns and job titles as rows. For the purpose of this project, jobs are filtered depending on their job zones. 
All the data can be found here:
https://www.onetcenter.org/database.html

In [2]:
# Load datasets into Pandas DataFrame
skills = pd.read_excel('../databases/ONET/Skills.xlsx')

In [3]:
knowledge = pd.read_excel('../databases/ONET/Knowledge.xlsx')

In [4]:
# Merge skills and knowledge files
df = skills.append(knowledge)

In [5]:
# Filter per job zones 3, 4 and 5
job_zones = pd.read_excel('../databases/ONET/Job Zones.xlsx')
jobzone345 = job_zones[job_zones['Job Zone'].isin([3,4,5])]['O*NET-SOC Code'].tolist()
df = df[df['O*NET-SOC Code'].isin(jobzone345)]

In [6]:
# Take out unnecessary columns
cols_to_keep = [
    'O*NET-SOC Code', 
    'Title', 
    'Element Name', 
    'Scale ID',
    'Data Value'
]

df = df[cols_to_keep].copy()

In [7]:
# Merge columns 'Element Name' and 'Scale ID' into 'element_scale_id'
df['element_scale_id'] = df['Element Name'] + '_' + df['Scale ID'] 

In [8]:
# Pivot table
df_pivot = df.pivot_table(index=['Title'], columns='element_scale_id', values='Data Value')

In [9]:
df_pivot.head()

element_scale_id,Active Learning_IM,Active Learning_LV,Active Listening_IM,Active Listening_LV,Administration and Management_IM,Administration and Management_LV,Biology_IM,Biology_LV,Building and Construction_IM,Building and Construction_LV,Chemistry_IM,Chemistry_LV,Clerical_IM,Clerical_LV,Communications and Media_IM,Communications and Media_LV,Complex Problem Solving_IM,Complex Problem Solving_LV,Computers and Electronics_IM,Computers and Electronics_LV,Coordination_IM,Coordination_LV,Critical Thinking_IM,Critical Thinking_LV,Customer and Personal Service_IM,Customer and Personal Service_LV,Design_IM,Design_LV,Economics and Accounting_IM,Economics and Accounting_LV,Education and Training_IM,Education and Training_LV,Engineering and Technology_IM,Engineering and Technology_LV,English Language_IM,English Language_LV,Equipment Maintenance_IM,Equipment Maintenance_LV,Equipment Selection_IM,Equipment Selection_LV,Fine Arts_IM,Fine Arts_LV,Food Production_IM,Food Production_LV,Foreign Language_IM,Foreign Language_LV,Geography_IM,Geography_LV,History and Archeology_IM,History and Archeology_LV,...,Persuasion_IM,Persuasion_LV,Philosophy and Theology_IM,Philosophy and Theology_LV,Physics_IM,Physics_LV,Production and Processing_IM,Production and Processing_LV,Programming_IM,Programming_LV,Psychology_IM,Psychology_LV,Public Safety and Security_IM,Public Safety and Security_LV,Quality Control Analysis_IM,Quality Control Analysis_LV,Reading Comprehension_IM,Reading Comprehension_LV,Repairing_IM,Repairing_LV,Sales and Marketing_IM,Sales and Marketing_LV,Science_IM,Science_LV,Service Orientation_IM,Service Orientation_LV,Social Perceptiveness_IM,Social Perceptiveness_LV,Sociology and Anthropology_IM,Sociology and Anthropology_LV,Speaking_IM,Speaking_LV,Systems Analysis_IM,Systems Analysis_LV,Systems Evaluation_IM,Systems Evaluation_LV,Technology Design_IM,Technology Design_LV,Telecommunications_IM,Telecommunications_LV,Therapy and Counseling_IM,Therapy and Counseling_LV,Time Management_IM,Time Management_LV,Transportation_IM,Transportation_LV,Troubleshooting_IM,Troubleshooting_LV,Writing_IM,Writing_LV
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Accountants,3.25,3.38,3.88,3.62,3.43,4.00,1.02,0.03,1.21,0.40,1.08,0.22,3.56,5.34,2.22,2.13,3.38,3.25,3.57,4.34,3.12,3.00,3.62,3.88,3.39,4.23,1.30,0.59,4.48,5.54,1.89,1.94,1.29,0.56,3.59,3.64,1.0,0.0,1.00,0.00,1.00,0.01,1.17,0.34,1.42,0.84,1.29,0.57,1.18,0.41,...,2.62,2.75,1.20,0.43,1.10,0.22,2.10,1.84,1.38,0.50,1.71,1.40,2.06,1.70,1.12,0.25,3.75,3.75,1.00,0.00,2.10,1.95,1.88,1.62,2.75,3.00,2.88,3.00,1.36,0.81,3.62,3.62,3.12,3.50,2.75,3.12,1.50,0.62,1.89,1.16,1.13,0.33,3.38,2.88,1.74,1.25,1.00,0.00,3.75,3.38
Actuaries,3.38,4.12,4.00,4.12,3.31,4.72,1.16,0.47,1.25,0.38,1.06,0.16,2.00,2.84,2.16,2.23,4.00,4.62,3.73,4.55,3.00,3.00,4.25,4.75,2.59,3.09,1.75,1.56,4.19,5.25,2.50,3.59,1.41,1.12,3.74,4.41,1.0,0.0,1.00,0.00,1.00,0.00,1.00,0.00,1.34,1.16,1.65,1.62,1.16,0.41,...,3.00,3.62,1.16,0.28,1.16,0.38,1.69,1.41,2.50,2.75,2.00,2.47,1.56,1.25,2.00,1.75,4.12,4.62,1.00,0.00,2.31,3.38,2.12,2.00,2.88,3.12,2.88,2.88,1.97,1.94,3.88,4.00,3.88,4.50,4.00,4.50,1.75,1.00,1.41,0.53,1.31,0.78,3.12,3.50,1.34,0.88,1.00,0.00,3.38,4.00
Acupuncturists,3.12,3.50,3.75,3.62,2.67,2.57,3.45,4.05,1.60,0.85,2.22,2.65,2.48,2.97,2.37,2.70,3.00,3.25,2.28,2.91,3.00,2.88,3.75,3.88,4.17,5.10,1.64,0.94,2.50,2.21,2.95,3.78,1.62,1.05,3.82,3.89,1.0,0.0,1.25,0.50,1.19,0.38,2.00,1.29,1.82,1.68,1.26,0.76,1.63,1.24,...,2.62,2.88,2.80,4.01,1.74,1.19,1.79,1.29,1.38,0.62,3.96,5.43,2.26,2.24,2.25,2.12,3.25,3.88,1.00,0.00,3.01,3.41,2.62,2.12,3.62,3.38,3.62,3.75,2.89,3.53,3.50,3.62,2.75,2.62,2.75,3.12,1.62,1.00,1.77,0.67,3.97,4.64,2.75,2.62,1.54,0.84,1.50,0.75,3.25,3.50
Acute Care Nurses,3.75,3.88,4.00,3.88,2.89,2.96,3.32,3.50,1.18,0.39,2.71,2.89,2.68,3.11,2.22,2.21,3.75,3.75,2.61,3.00,3.75,3.88,4.00,4.00,4.36,5.61,1.36,0.64,1.68,1.11,4.14,4.71,1.68,1.21,4.29,4.46,1.0,0.0,1.88,1.00,1.18,0.21

In [19]:
# Display size of the matrix
df_pivot.shape

(640, 134)

Let's factorize this matrix to (640, 2)

# Create the embedding with t-SNE

We create an embedding with t-SNE and project it in 2D.

In [20]:
# Set up
import numpy as np
from sklearn.manifold import TSNE

In [21]:
tsne = TSNE(n_components=2)

In [23]:
embedding  = tsne.fit_transform(df_pivot)

In [27]:
embedding.shape

(640, 2)

In [28]:
embedding

array([[-19.515848 ,  -4.0266066],
       [-21.757238 ,  -7.4855747],
       [ -6.903803 ,  26.09269  ],
       ...,
       [  7.74189  ,  -2.1080961],
       [ 16.597858 , -24.651657 ],
       [ 15.132826 ,   5.426909 ]], dtype=float32)

We now have an array of floats that correspond to the coordinates of each point of the dataset in the target space. Let's visualize it.

# Visualization with plotly

In [30]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [32]:
data = [
    go.Scatter(
        x=embedding[:,0],
        y=embedding[:,1],
        mode='markers',
        text=df_pivot.index.tolist(),
        opacity=0.5
    )
]
layout = go.Layout(
    title='Jobs similarity map (based on skills and knowledge) with t-SNE',
    hovermode='closest'
)
fig = go.Figure(data=data, layout=layout)
plot_url = iplot(fig, filename='hover-chart-basic')